In [1]:
import keras_tuner
import numpy as np
import sklearn.metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import Modules.constants as constants
import Modules.ds_loader as ds_loader

SAMPLE_PERCENTAGE = 1.0

DATA_PATH = constants.DATASET
TRAIN_DIR = DATA_PATH / "train"
VAL_DIR = DATA_PATH / "val"
TEST_DIR = DATA_PATH / "test"

X_train, y_train = ds_loader.load_all_data(TRAIN_DIR, SAMPLE_PERCENTAGE)
X_val, y_val = ds_loader.load_all_data(VAL_DIR, SAMPLE_PERCENTAGE)
X_test, y_test = ds_loader.load_all_data(TEST_DIR, SAMPLE_PERCENTAGE)

2025-04-01 21:51:38.235308: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 21:51:38.243840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743537098.253567  100050 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743537098.256352  100050 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743537098.264444  100050 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
print("Unique classes in y:", np.unique(y_train))
print("Datatype:", print(X_train.dtype), print(y_train.dtype))
print(X_train.shape, X_test.shape, X_val.shape)
print(y_train.shape, y_test.shape, y_val.shape)
print(f"Min and Max of X_train: {np.min(X_train)}, {np.max(X_train)}")
print(f"Min and Max of X_test: {np.min(X_test)}, {np.max(X_test)}")
print(f"Min and Max of X_val: {np.min(X_val)}, {np.max(X_val)}")
print(f"NaNs in X: {np.isnan(X_train).sum()}")
print(f"Infs in X: {np.isinf(X_train).sum()}")

Unique classes in y: [0 1 2 3]
float32
int32
Datatype: None None
(8502, 10, 12) (532, 10, 12) (1594, 10, 12)
(8502,) (532,) (1594,)
Min and Max of X_train: 0.0, 1.0000009536743164
Min and Max of X_test: -94.44111633300781, 192.52906799316406
Min and Max of X_val: -223.99363708496094, 127.48966979980469
NaNs in X: 0
Infs in X: 0


In [3]:
# 1-D convolutional ResNet model 
# https://pmc.ncbi.nlm.nih.gov/articles/PMC10128986/#sec012
class Resnet(keras_tuner.HyperModel):
    def residual_block(self, inputs, filters, kernel_size):
        shortcut = tf.keras.layers.Conv1D(filters=filters, kernel_size=1, padding='same')(inputs)
        
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(inputs)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # SKIP CONNECTION
        x = tf.keras.layers.Add()([x, shortcut]) 
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling1D(pool_size=5, strides=2)(x)
        
        return x

    def build(self, hp):
        c_units = hp.Int('c_units', min_value=4, max_value=64, step=2)
        d_units = hp.Int('d_units', min_value=4, max_value=128, step=2)
        dropout = hp.Float('dropout', min_value = 0.4, max_value=0.5)
        lr = hp.Float('lr', min_value=0.00001, max_value=0.001)
        n = hp.Choice('#_res_layers', [1,2,3])
        k_units = hp.Int('k_units', min_value=3, max_value=15, step=1)
        m = hp.Float('momentum', min_value=0.1, max_value=0.9, step=0.1)
        # INPUT LAYER
        inputs = tf.keras.Input(shape=(500,12))
        
        # RESIDUALS
        x = self.residual_block(inputs, filters=c_units, kernel_size=k_units)
        for _ in range(n):  
            x = self.residual_block(x, filters=c_units, kernel_size=k_units)
        
        # CLASSIFIER
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(d_units, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)  

        x = tf.keras.layers.Dense(d_units // 2, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)  

        # OUTPUT
        outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

        model = tf.keras.Model(inputs, outputs)
        
        model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=m),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            epochs = hp.Int("epochs", 10, 150, 10),
            batch_size= hp.Choice("batch_size", [16, 32, 64]),
            *args,
            **kwargs,
        )

Trial 15 summary
Hyperparameters:
c_units: 16
d_units: 16
dropout: 0.4
lr: 0.007954238351337758
#_res_layers: 3
k_units: 9
epochs: 80
batch_size: 16
Score: 0.409099817276001

In [4]:
tuner = keras_tuner.RandomSearch(
    Resnet(),
    max_trials=100,
    objective='val_loss',
    directory="Results/RESNET",
    project_name="ECGClassification",
    )
tuner.search_space_summary()

Reloading Tuner from Results/RESNET/ECGClassification/tuner0.json
Search space summary
Default search space size: 9
c_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 64, 'step': 2, 'sampling': 'linear'}
d_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 128, 'step': 2, 'sampling': 'linear'}
dropout (Float)
{'default': 0.4, 'conditions': [], 'min_value': 0.4, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
lr (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'linear'}
#_res_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
k_units (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 15, 'step': 1, 'sampling': 'linear'}
momentum (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.9, 'step': 0.1, 'sampling': 'linear'}
epochs (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_v

In [5]:
callback_list = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=0)
]

tuner.search(
    X_train, y_train, 
    validation_data=(X_val, y_val),
    callbacks=callback_list 
)


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
38                |38                |c_units
78                |78                |d_units
0.45305           |0.45305           |dropout
0.00056706        |0.00056706        |lr
3                 |3                 |#_res_layers
5                 |5                 |k_units
0.3               |0.3               |momentum
90                |90                |epochs
32                |32                |batch_size



I0000 00:00:1743537102.354780  100050 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1347 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/90


Traceback (most recent call last):
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capitan/.venv/tenv/lib64/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.f

In [ ]:
tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()
best_model.save("Results/BEST_RESNET_00.h5") 

# RESNET50 WITH BOTTLENECK

In [ ]:
"""
class Resnet50(tf.keras.Model):
    
    def bottleneck_block(self, inputs, filters=16, strides=1):
        # First convolution (1x1)
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=1, strides=strides, padding='same')(inputs)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # Second convolution (3x3)
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # Third convolution (1x1)
        x = tf.keras.layers.Conv1D(filters=filters * 4, kernel_size=1, padding='same')(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # Shortcut path (1x1 convolution if necessary)
        shortcut = tf.keras.layers.Conv1D(filters=filters * 4, kernel_size=1, strides=strides, padding='same')(inputs)
        
        # Add the shortcut to the residual output
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        return x

    def build(self, hp):
        c_units = hp.Int('c_units', min_value=16, max_value=64, step=16)  # Increased filter size for ResNet50-like architecture
        d_units = hp.Int('d_units', min_value=64, max_value=256, step=64)
        dropout = hp.Float('dropout', min_value=0.3, max_value=0.5, step=0.1)
        lr = hp.Float('lr', min_value=1e-4, max_value=1e-2)
        n = hp.Choice('#_res_layers', [2, 3, 4])  # You can experiment with the number of residual layers
        
        # INPUT LAYER
        inputs = tf.keras.Input(shape=(5000, 12))  # Shape of input

        # First Convolution (For initial feature extraction)
        x = tf.keras.layers.Conv1D(filters=c_units, kernel_size=7, strides=2, padding='same')(inputs)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        # First Residual Block (typically after initial conv)
        x = self.bottleneck_block(x, filters=c_units)
        
        # Additional Residual Blocks (stacked)
        for _ in range(n):  
            x = self.bottleneck_block(x, filters=c_units)

        # Global Average Pooling (typically used in ResNet50)
        x = tf.keras.layers.GlobalAveragePooling1D()(x)
        
        # CLASSIFIER
        x = tf.keras.layers.Dense(d_units, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)
        
        x = tf.keras.layers.Dense(d_units // 2, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout)(x)

        # OUTPUT
        outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

        model = tf.keras.Model(inputs, outputs)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            epochs = hp.Int("epochs", 50, 100, 10),
            batch_size= hp.Choice("batch_size", [4, 8, 16, 32]),
            *args,
            **kwargs,
        )"""

In [ ]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, batch_size=32)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  
auc = sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

print("Classification Report (Test Data):")
print(sklearn.metrics.classification_report(y_test, y_pred))
print(f"AUC: {auc}")

y_train_pred = best_model.predict(X_train)
y_train_pred = np.argmax(y_train_pred, axis=1)

print("Classification Report (Train Data):")
print(sklearn.metrics.classification_report(y_train, y_train_pred))


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(best_hps.values)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
modified_hps = best_hps.copy()

#modified_hps.values['c_units'] = 16
#modified_hps.values['k_units'] = 5 
#best_model = Resnet().build(modified_hps)

#best_model = Resnet().build(best_hps)
#best_model = RNN().build(best_hps)
best_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.5),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy']) 
history = best_model.fit(
    X_train, y_train, 
    validation_data=(X_val, y_val),
    epochs=150,
    batch_size=16
    )


In [ ]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, batch_size=16)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  
auc = sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

print("Classification Report (Test Data):")
print(sklearn.metrics.classification_report(y_test, y_pred))
print(f"AUC: {auc}")

y_train_pred = best_model.predict(X_train)
y_train_pred = np.argmax(y_train_pred, axis=1)

print("Classification Report (Train Data):")
print(sklearn.metrics.classification_report(y_train, y_train_pred))

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].plot(history.history['accuracy'], label='accuracy')
ax[0].plot(history.history['val_accuracy'], label='val_accuracy')
ax[0].set_title('Accuracy vs Val Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend()

ax[1].plot(history.history['loss'], label='loss')
ax[1].plot(history.history['val_loss'], label='val_loss')
ax[1].set_title('Loss vs Val Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend()

plt.tight_layout()
plt.show()